# Projet TensorFlow - Classification d'images 
## SALMON Loïck et HENDRICÉ Rémy

## 1.2 Description du dataset

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds 
import matplotlib

In [ ]:
(training_dataset, validation_dataset), info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

### 2. Description du DataSet

In [ ]:
tfds.show_examples(training_dataset, info)
len(training_dataset) + len(validation_dataset)
len(training_dataset)
len(validation_dataset)

Nous avons 23262 images pour l'apprentissage. Ce sont des images rectangulaires en couleurs.
Il y a deux classes : chien (1) ou chat (0)

### 3. Déterminer les paramètre de taille d'image, du nombre de neurons en entrée et la taille du batch

Pour cette exercice, nous avons choisi d'utiliser des images carrées car cela simplifie le traitement. Nous avons choisi une taille de 128x128 pixels.

Le nombre d'entrées est directement lié à la taille des images. Chaque valeur R,G,B correspond à une entrée. Il y a donc 128\*128\*3 = 49152 entrées.

Pour le batch, nous avons choisi une taille de 32. Cela permet de ne pas surcharger la mémoire tout en permettant un apprentissage efficace.

### 4. Répartition du dataset

Nous avons dédié 80% du dataset à l'entraînement et 20% à la validation comme conseillé dans la documentation TensorFlow https://www.tensorflow.org/tutorials/images/classification#load_data_using_a_keras_utility 

> It's good practice to use a validation split when developing your model. Use 80% of the images for training and 20% for validation.



### 5. Normalisation des données

Nous pouvons redimensionner nos images avec une couche Resizing ddans notre modèle

In [ ]:
resizing_layer = tf.keras.layers.Resizing(128,128)

Nous utilisons une couche Rescaling pour normaliser les valeurs de pixels de 0 à 255 à un intervalle de 0 à 1.


In [ ]:
rescaling_layer = tf.keras.layers.Rescaling(1./255, input_shape=(128,128,3))

### 6. Création du réseau de neurones

Pour le réseau de neurones, nous allons appliquer nos couches de resizing et de rescaling puis nous allons appliquer notre réseau de convolution.

In [ ]:
model = tf.keras.Sequential([
    resizing_layer,
    rescaling_layer,
    tf.keras.layers.Conv2D(16,3,padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32,3,padding='same', activation='relu'),
])

In [ ]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))

De là on constuit le modèle

In [ ]:
model.compile(optimizer = 'adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
              )


In [ ]:
history = model.fit(training_dataset, epochs=10, 
                    validation_data=validation_dataset)


### 7. Discussion des résultats obtenus

### 8. Image perso 